In [46]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.formula.api import logit
from geopy.geocoders import Nominatim

import pylab as pl
import seaborn as sns

mpl.style.use('fivethirtyeight')
%matplotlib inline

In [91]:
crime = pd.read_csv('https://raw.githubusercontent.com/jjpowe13/Drunk_Crimes/master/Drunk_Crime_Reports.csv')
crime = crime.drop(crime.columns[3:21], axis=1)
crime = crime.drop(crime.columns[0:2], axis=1)
crime = crime[crime.Latitude_Fixed != 0]
crime = crime.reset_index(drop=True)
crime['Zip'] = ''
crime

,INCIDENT_TYPE_DESCRIPTION,Latitude_Fixed,Longitude_Fixed,Zip
0,DISORDERLY,42.314001,-71.086340,
1,DISORDERLY,42.288621,-71.075580,
2,DISORDERLY,42.251951,-71.125320,
3,DISORDERLY,42.328021,-71.092370,
4,DISORDERLY,42.306752,-71.060280,
5,DISORDERLY,42.246833,-71.124050,
6,DISORDERLY,42.317592,-71.040034,
7,DISORDERLY,42.315621,-71.098470,
8,DISORDERLY,42.300521,-71.064945,
9,DISORDERLY,42.335766,-71.074080,


In [76]:
def findzip(x,y):
    geolocator = Nominatim()
    location = [x, y]
    location = str(location)
    location = location.strip('[]')
    location = geolocator.reverse(location)
    location = str(location.raw)
    location = location.split(',')
    zip = location[7].strip()
    return zip

In [77]:
test = crime.head(5)
test['Zip'] = ''

  INCIDENT_TYPE_DESCRIPTION  Latitude_Fixed  Longitude_Fixed Zip
0                DISORDERLY       42.314001        -71.08634    
1                DISORDERLY       42.288621        -71.07558    
2                DISORDERLY       42.251951        -71.12532    
3                DISORDERLY       42.328021        -71.09237    
4                DISORDERLY       42.306752        -71.06028    


/Users/ayellis/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [78]:
for i in test.index:
    test.ix[i, 'Zip'] = findzip(test.Latitude_Fixed[i], test.Longitude_Fixed[i])
print test

  INCIDENT_TYPE_DESCRIPTION  Latitude_Fixed  Longitude_Fixed    Zip
0                DISORDERLY       42.314001        -71.08634  02121
1                DISORDERLY       42.288621        -71.07558  02124
2                DISORDERLY       42.251951        -71.12532  02136
3                DISORDERLY       42.328021        -71.09237  02119
4                DISORDERLY       42.306752        -71.06028  02122


# The miracle is below~

In [79]:
for i in crime.index:
    crime.ix[i, 'Zip'] = findzip(crime.Latitude_Fixed[i], crime.Longitude_Fixed[i])
print crime.sample(5)

GeocoderServiceError: <urlopen error [Errno 65] No route to host>

In [ ]:
crime = pd.read_csv('https://raw.githubusercontent.com/jjpowe13/Drunk_Crimes/master/Drunk_Crime_Reports2.csv')
crime = crime.drop(crime.columns[3:21], axis=1)
crime = crime.drop(crime.columns[0:2], axis=1)

Next step - set the 'geolocate' function to be able to read in the data frame...

## Did nothing below here.

In [127]:
#x = pd.concat([crime[['Latitude_Fixed']], crime[['Longitude_Fixed']]])
#crime2 = np.insert(crime.Latitude_Fixed, crime.Longitude_Fixed, axis=1)
#crime2 = crime2.join(crime.Longitude_Fixed)
#crime2 = crime.loc[:,crime.columns.tolist()[1:]].apply(lambda x: x.dropna().tolist(), 1)
#crime['geolocation'] = crime.merge([[crime.Latitude_Fixed], [crime.Longitude_Fixed]])
#crime['combined'] = 

#df = pd.concat([df1[['value']], df2[['value']]])
#crime2

In [ ]:
#test= pd.read_csv('https://raw.githubusercontent.com/jjpowe13/Drunk_Crimes/master/Drunk_Crime_Reports.csv')
#test = test.drop(test.columns[3:19], axis=1)
#test = test.drop(test.columns[[0,1,4,5]], axis=1)
#test = test.reset_index(drop=True)
#test.head()

In [ ]:
# test['zip'] = geocoder.google(test['Geo']).postal
#mypivot = pd.pivot_table(aisehi,index=['zip'], values=['INCIDENT_TYPE_DESCRIPTION'], aggfunc = lambda x: len(x.unique()))
#mypivot
# test

In [ ]:
import geocoder
test['zip'] = geocoder.google(aisehi['Geo'].all()).postal

test

In [ ]:
#>>> import geocoder
#>>> crime['zip'] = geocoder.google(crime['Latitude_Fixed'], crime['Longitude_Fixed']).postal
#crime
#crime.Longitude_Fixed